# Circuit Decomposition 
Edge and gate decomposition with zero classical communication.

Prerequisites: 

1. Create a new Conda environment
```shell
conda env create -n knitting
```

1. Install everything needed to run the `circuit-knitting-toolbox` locally: 
```shell
cd circuit-knitting-toolbox
pip install -e .
```


In [1]:
# Prevent from overriding already run experiments
# exit()

In [1]:
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import EfficientSU2

# Create a quantum circuit to cut. We create a simple ansatz
mapper = JordanWignerMapper()
ansatz = EfficientSU2(3, reps=1)
# Decompose to the actual individual gates
circuit = ansatz.decompose(reps=3)
# Set some arbitrary parameters
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)

print(circuit)

global phase: 3.8832
     ┌─────────────┐┌─────────────┐                    ┌─────────────┐»
q_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├────────────■───────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤          ┌─┴─┐     ├─────────────┤»
q_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──■───────┤ X ├─────┤ U3(0.8,0,0) ├»
     ├─────────────┤├─────────────┤┌─┴─┐┌────┴───┴────┐├─────────────┤»
q_2: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ X ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├»
     └─────────────┘└─────────────┘└───┘└─────────────┘└─────────────┘»
«     ┌─────────────┐
«q_0: ┤ U3(0,0,0.8) ├
«     ├─────────────┤
«q_1: ┤ U3(0,0,0.8) ├
«     └─────────────┘
«q_2: ───────────────
«                    


Create a list of observables to use, and automatically find cut locations.

In [2]:
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import cut_wires_and_gates_to_subcircuits

observables = ["ZZI", "IZZ", "IIZ", "XIX", "ZIZ", "IXI"]

subcircuits, subobservables = cut_wires_and_gates_to_subcircuits(
    circuit=circuit,
    observables=observables,
    method='automatic',
    max_subcircuit_width=2,
    max_cuts=9,
    num_subcircuits=[2],
    model='gurobi'
)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 2 times.
MIP Presolve eliminated 19 rows and 7 columns.
MIP Presolve modified 3 coefficients.
Aggregator did 24 substitutions.
Reduced MIP has 11 rows, 8 columns, and 35 nonzeros.
Reduced MIP has 5 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.12 ticks)
Found incumbent of value 2.000000 after 0.02 sec. (0.13 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 11 rows, 8 columns, and 35 nonzeros.
Reduced MIP has 5 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 1.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic

In [4]:
# Visualize the subcircuits. Note the decomposed 2-qubit gates marked 'cut_cx_0'
for key in subcircuits.keys():
    print(subcircuits[key])

      ┌─────────────┐┌─────────────┐                 ┌─────────────┐»
q1_0: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├──────────────■──┤ U3(0.8,0,0) ├»
      ├─────────────┤├─────────────┤┌──────────┐┌─┴─┐├─────────────┤»
q1_1: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ X ├┤ U3(0.8,0,0) ├»
      └─────────────┘└─────────────┘└──────────┘└───┘└─────────────┘»
«      ┌─────────────┐
«q1_0: ┤ U3(0,0,0.8) ├
«      ├─────────────┤
«q1_1: ┤ U3(0,0,0.8) ├
«      └─────────────┘
    ┌─────────────┐┌─────────────┐┌──────────┐┌─────────────┐┌─────────────┐
q2: ┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├┤ cut_cx_0 ├┤ U3(0.8,0,0) ├┤ U3(0,0,0.8) ├
    └─────────────┘└─────────────┘└──────────┘└─────────────┘└─────────────┘


In [5]:
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import execute_simulation

# Execute the subcircuits
quasi_dists, coefficients = execute_simulation(subcircuits, subobservables)

Now check the actual expectation values from the observables.

In [6]:
from circuit_knitting.cutting.cutting_reconstruction import reconstruct_expectation_values

simulation_expvals = reconstruct_expectation_values(quasi_dists, coefficients, subobservables)
simulation_expvals

[0.515217661857605,
 0.5724585056304932,
 0.39968442916870117,
 0.09736669063568115,
 0.19814372062683105,
 0.24010205268859863]

In [7]:
# Create ideal results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import exact_observables

ideal_expvals = exact_observables(circuit, observables)
ideal_expvals

array([0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229,
       0.23530298])

In [9]:
# Compare the error between results
from circuit_knitting.cutting.gate_and_wire_cutting.frontend import compare_results

compare_results(simulation_expvals, ideal_expvals)

Simulated expectation values: [0.51521766, 0.57245851, 0.39968443, 0.09736669, 0.19814372, 0.24010205]
Exact expectation values: [0.50390696, 0.57454157, 0.39584538, 0.09798816, 0.18481229, 0.23530298]
Errors in estimation: [0.0113107, -0.00208307, 0.00383905, -0.00062147, 0.01333143, 0.00479908]
Relative errors in estimation: [0.022446, -0.00362561, 0.00969835, -0.00634232, 0.07213496, 0.02039531]
